In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook
import pandas as pd
import numpy as np
import seaborn as sns
import json
from pathlib import Path
from scipy.stats import mannwhitneyu
sns.set(style='ticks', palette='deep')
%load_ext autoreload
%autoreload 2

### loading, changing/combining the fish dfs

In [ ]:
#load all fish df
all_fish= pd.read_json("../data/population_data.json")
all_fish

In [ ]:
# if i need to combine multiple data frames, for example, several batches

#load all dfs first:
all_fish1= pd.read_json(r'')
all_fish2= pd.read_json(r'')
#... add mroe dfs if needed

#combine all dfs into the all_fish df: (add more as needed)
all_fish=pd.concat([all_fish1, all_fish2], ignore_index=True)
all_fish

In [ ]:
#this is the path where i save all figs/updated df
path_save = Path(r'')

In [ ]:
#save updated/combined df
all_fish.to_json(str(path_save)+'/updated_all_pool.json')

### some functions and variables 

In [ ]:
wt_col='#5D5D5D'
mut_col='#BE4830'
#need to change here depending on the genotype
pal= {"wt": '#5D5D5D', "inj":'#BE4830'}

In [ ]:
def set_box_color(bp, color):
    plt.setp(bp['boxes'], facecolor=color, alpha=0.5)
    plt.setp(bp['whiskers'], color=color)
    plt.setp(bp['caps'], color=color)
    plt.setp(bp['medians'], color=color)

    
# getting all angles from various stim times (either bins or whole stim1 vs stim2)
def get_angles(df):
    stim_p_angles=[]
    stim_r_angles=[]
    stim_l_angles=[]

    for i in range(len(df)):
        stim_p_angles.append(df.angles_pause.iloc[i])
        stim_r_angles.append(df.angles_stim_r.iloc[i])
        stim_l_angles.append(df.angles_stim_l.iloc[i])

        #unpack from the list of angle arrays    
    angles_p=[item for sublist in stim_p_angles for item in sublist]
    angles_r=[item for sublist in stim_r_angles for item in sublist]
    angles_l=[item for sublist in stim_l_angles for item in sublist]
    return (angles_p, angles_r, angles_l) 


def plot_params(param_1_df1, param_1_df2, param_2_df1, param_2_df2):

    stim1_1=ax[0].boxplot(param_1_df1, positions=[1], sym='', widths=0.6, patch_artist=True)
    stim1_2=ax[0].boxplot(param_1_df2, positions=[2], sym='', widths=0.6, patch_artist=True)

    stim2_1=ax[1].boxplot(param_2_df1, positions=[1], sym='', widths=0.6, patch_artist=True)
    stim2_2=ax[1].boxplot(param_2_df2, positions=[2], sym='', widths=0.6, patch_artist=True)

    set_box_color(stim1_1, wt_col) 
    set_box_color(stim1_2, mut_col) 
    set_box_color(stim2_1, wt_col)
    set_box_color(stim2_2, mut_col)

    ax[0].set_xticklabels(['wt', 'mutant'], size=14)
    ax[0].set_title('During Pause')

    ax[1].set_xticklabels(['wt', 'mutant'], size=14)
    fig.subplots_adjust(wspace=0.2, hspace=0.3)
    ax[1].set_title('During stimulus')
    #plt.text(7.1, 40, 'n = '+str(len(all_fish_new)), fontsize=12)
    sns.despine()

In [ ]:
#dividing the df into mutant vs wt (sometimes need to change 'genotype' with 'comments'- adjust based on the mutant tested each time)
df1=all_fish[all_fish['genotype']=='wt'].reset_index(drop=True)
df2=all_fish[all_fish['genotype']=='inj'].reset_index(drop=True)

### Plotting 

In [ ]:
#File where all p values and test are saved
file_path= path_save / 'p_values_results.txt'

with open(file_path, "w") as f:
    f.write("P-values for each metric:\n\n")

In [ ]:
# plotting total number of bouts (including forward swims) separated by stims and groups

n_total_pause_df1=df1.n_left_pause+df1.n_right_pause+df1.n_forward_pause
n_total_right_df1=df1.n_left_stim_r+df1.n_right_stim_r+df1.n_forward_stim_r
n_total_left_df1=df1.n_left_stim_l+df1.n_right_stim_l+df1.n_forward_stim_l

n_total_pause_df2=df2.n_left_pause+df2.n_right_pause+df2.n_forward_pause
n_total_right_df2=df2.n_left_stim_r+df2.n_right_stim_r+df2.n_forward_stim_r
n_total_left_df2=df2.n_left_stim_l+df2.n_right_stim_l+df2.n_forward_stim_l

ticks=['pause', 'right stimulus','left stimulus']


n_total_ct= [n_total_pause_df1, n_total_right_df1, n_total_left_df1]
n_total_mut= [n_total_pause_df2, n_total_right_df2, n_total_left_df2]

fig, ax=plt.subplots( figsize=(5,5))

fr_ct=ax.boxplot(n_total_ct, positions=np.array(range(3))*2.0-0.4, sym='', widths=0.6, patch_artist=True)
fr_mut=ax.boxplot(n_total_mut, positions=np.array(range(3))*2.0+0.4, sym='', widths=0.6, patch_artist=True)

set_box_color(fr_ct, wt_col) 
set_box_color(fr_mut, mut_col)

ax.set_ylabel('Total number of swim bouts', size=14)
#ax.set_title('Total number of bouts including forward swims', size=14)
ax.set_xticks([0,2, 4])
ax.set_xticklabels(ticks, size=10)

ax.set_yticks([0, 25, 50, 75, 100, 125, 150])  # change here if need to adjust y axis limits
ax.legend([fr_ct["boxes"][0], fr_mut["boxes"][0]], ['wt', 'mutants'], loc='lower right',  bbox_to_anchor=(1.4,1.1))
plt.tight_layout()

sns.despine()
plt.savefig(str(path_save) +'/total_number_of_bouts_w_forward.jpeg', bbox_inches='tight')

print('pause: ', mannwhitneyu (n_total_ct[0], n_total_mut[0]))
print('right stim: ', mannwhitneyu (n_total_ct[1], n_total_mut[1]))
print('left stim: ', mannwhitneyu (n_total_ct[2], n_total_mut[2]))
# with open(file_path, "a") as f:
#     f.write(f"Results for total number of bouts (including forward swims):\n")
#     f.write(f"pause: statistic={mannwhitneyu(n_total_ct[0], n_total_mut[0]).statistic}, p-value={mannwhitneyu(n_total_ct[0], n_total_mut[0]).pvalue}\n")
#     f.write(f"right stim: statistic={mannwhitneyu(n_total_ct[1], n_total_mut[1]).statistic}, p-value={mannwhitneyu(n_total_ct[1], n_total_mut[1]).pvalue}\n")
#     f.write(f"left stim: statistic={mannwhitneyu(n_total_ct[2], n_total_mut[2]).statistic}, p-value={mannwhitneyu(n_total_ct[2], n_total_mut[2]).pvalue}\n\n")

In [ ]:
# fr correct just between left and right
fr_corr_new_stim_r_wt=[]
fr_corr_new_stim_l_wt=[]
fr_corr_new_stim_r_mut=[]
fr_corr_new_stim_l_mut=[]

for i in range(len(df1)):
    fr_corr_new_stim_r_wt.append(df1.iloc[i].n_right_stim_r/(df1.iloc[i].n_right_stim_r+df1.iloc[i].n_left_stim_r))
    fr_corr_new_stim_l_wt.append(df1.iloc[i].n_left_stim_l/(df1.iloc[i].n_right_stim_l+df1.iloc[i].n_left_stim_l))

for j in range(len(df2)):
    fr_corr_new_stim_r_mut.append(df2.iloc[j].n_right_stim_r/(df2.iloc[j].n_right_stim_r+df2.iloc[j].n_left_stim_r))
    fr_corr_new_stim_l_mut.append(df2.iloc[j].n_left_stim_l/(df2.iloc[j].n_right_stim_l+df2.iloc[j].n_left_stim_l))
    
ticks=['right stimulus','left stimulus']


fr_new_c_ct= [fr_corr_new_stim_r_wt, fr_corr_new_stim_l_wt]
fr_new_c_mut= [fr_corr_new_stim_r_mut,fr_corr_new_stim_l_mut]

fig, ax=plt.subplots( figsize=(5,5))

fr_ct=ax.boxplot(fr_new_c_ct, positions=np.array(range(2))*2.0-0.4, sym='', widths=0.6, patch_artist=True)
fr_mut=ax.boxplot(fr_new_c_mut, positions=np.array(range(2))*2.0+0.4, sym='', widths=0.6, patch_artist=True)

set_box_color(fr_ct, wt_col) 
set_box_color(fr_mut, mut_col)

ax.set_ylabel('Fraction correct turns', size=14)
ax.set_xticks([0,2])
ax.set_xticklabels(ticks, size=12)
#ax.set_title('Fraction correct turns (turns only)', size=14)
ax.set_yticks([0,0.25, 0.5,0.75,1])
ax.legend([fr_ct["boxes"][0], fr_mut["boxes"][0]], ['wt', 'mutant'], loc='lower right',  bbox_to_anchor=(1.2,1.1))
plt.tight_layout()

sns.despine()
#plt.savefig(str(path_save) +'/fraction_correct_turns.jpeg', bbox_inches='tight')
print('right stimulus: ', mannwhitneyu (fr_corr_new_stim_r_wt,fr_corr_new_stim_r_mut))
print('left stimulus: ', mannwhitneyu (fr_corr_new_stim_l_wt,fr_corr_new_stim_l_mut))

# with open(file_path, "a") as f:
#     f.write(f"Results for the fraction of correct turns (stimulus only):\n")
#     f.write(f"right stim: statistic={mannwhitneyu(fr_corr_new_stim_r_wt,fr_corr_new_stim_r_mut).statistic}, p-value={mannwhitneyu(fr_corr_new_stim_r_wt,fr_corr_new_stim_r_mut).pvalue}\n")
#     f.write(f"left stim: statistic={mannwhitneyu(fr_corr_new_stim_l_wt,fr_corr_new_stim_l_mut).statistic}, p-value={mannwhitneyu(fr_corr_new_stim_l_wt,fr_corr_new_stim_l_mut).pvalue}\n\n")

In [ ]:
# mean bout displacement
fig, ax=plt.subplots(1, 2, figsize=(5,5), sharey=True)
plot_params(df1.mean_disp_pause.dropna(), df2.mean_disp_pause.dropna(), df1.mean_disp_stim.dropna(), df2.mean_disp_stim.dropna())
ax[0].set_ylabel('Mean bout displacement (mm)', size=14)
plt.savefig(str(path_save) +'/mean_bdisp.jpeg', bbox_inches='tight')
print ('during pause: ', mannwhitneyu (df1.mean_disp_pause.dropna(), df2.mean_disp_pause.dropna()))
print ('during stim: ', mannwhitneyu (df1.mean_disp_stim.dropna(), df2.mean_disp_stim.dropna()))

# with open(file_path, "a") as f:
#     f.write(f"Results for mean bout displacement:\n")
#     f.write(f"during pause: statistic={mannwhitneyu(df1.mean_disp_pause.dropna(), df2.mean_disp_pause.dropna()).statistic}, p-value={mannwhitneyu(df1.mean_disp_pause.dropna(), df2.mean_disp_pause.dropna()).pvalue}\n")
#     f.write(f"during stimulus: statistic={mannwhitneyu(df1.mean_disp_stim.dropna(), df2.mean_disp_stim.dropna()).statistic}, p-value={mannwhitneyu(df1.mean_disp_stim.dropna(), df2.mean_disp_stim.dropna()).pvalue}\n\n")

In [ ]:
# mean interbout interval
fig, ax=plt.subplots(1, 2, figsize=(5,5), sharey=True)
plot_params(df1.mean_ibi_pause.dropna(), df2.mean_ibi_pause.dropna(), df1.mean_ibi_stim.dropna(), df2.mean_ibi_stim.dropna())
ax[0].set_ylabel('Mean interbout interval (s)', size=14)
plt.savefig(str(path_save) +'/interbout_int.jpeg', bbox_inches='tight')
print ('during pause: ', mannwhitneyu (df1.mean_ibi_pause.dropna(), df2.mean_ibi_pause.dropna()))
print ('during stim: ', mannwhitneyu (df1.mean_ibi_stim.dropna(), df2.mean_ibi_stim.dropna()))

# with open(file_path, "a") as f:
#     f.write(f"Results for interbout interval:\n")
#     f.write(f"during pause: statistic={mannwhitneyu(df1.mean_ibi_pause.dropna(), df2.mean_ibi_pause.dropna()).statistic}, p-value={mannwhitneyu(df1.mean_ibi_pause.dropna(), df2.mean_ibi_pause.dropna()).pvalue}\n")
#     f.write(f"during stimulus: statistic={mannwhitneyu(df1.mean_ibi_stim.dropna(), df2.mean_ibi_stim.dropna()).statistic}, p-value={mannwhitneyu(df1.mean_ibi_stim.dropna(), df2.mean_ibi_stim.dropna()).pvalue}\n\n")

In [ ]:
fig, ax=plt.subplots(1, 2, figsize=(5,5), sharey=True)
plot_params(df1.mean_bdur_pause.dropna(), df2.mean_bdur_pause.dropna(), df1.mean_bdur_stim.dropna(), df2.mean_bdur_stim.dropna())
ax[0].set_ylabel('Mean bout duration (s)', size=14)
plt.savefig(str(path_save) +'/mean_bdur.jpeg', bbox_inches='tight')
print ('during pause: ', mannwhitneyu (df1.mean_bdur_pause.dropna(), df2.mean_bdur_pause.dropna()))
print ('during stim: ', mannwhitneyu (df1.mean_bdur_stim.dropna(), df2.mean_bdur_stim.dropna()))
# with open(file_path, "a") as f:
#     f.write(f"Results for mean bout duration:\n")
#     f.write(f"during pause: statistic={mannwhitneyu(df1.mean_bdur_pause.dropna(), df2.mean_bdur_pause.dropna()).statistic}, p-value={mannwhitneyu(df1.mean_bdur_pause.dropna(), df2.mean_bdur_pause.dropna()).pvalue}\n")
#     f.write(f"during stimulus: statistic={mannwhitneyu(df1.mean_bdur_stim.dropna(), df2.mean_bdur_stim.dropna()).statistic}, p-value={mannwhitneyu(df1.mean_bdur_stim.dropna(), df2.mean_bdur_stim.dropna()).pvalue}\n\n")

In [ ]:
vel_df1=[]
vel_df2=[]

for i in range(len(df1)):
    vel_df1.append(np.mean(df1.iloc[i]['mean_vel']))

for i in range(len(df2)):
    vel_df2.append(np.mean(df2.iloc[i]['mean_vel']))
    

fig, ax=plt.subplots(figsize=(3,5), sharey=True)
stim1_1=ax.boxplot(vel_df1, positions=[1], sym='', widths=0.2, patch_artist=True)
stim1_2=ax.boxplot(vel_df2, positions=[2], sym='', widths=0.2, patch_artist=True)


set_box_color(stim1_1, wt_col) 
set_box_color(stim1_2, mut_col) 
ax.set_xticklabels(['wt', 'mutant'], size=14)
sns.despine()
ax.set_ylabel('Mean bout velocity (mm/s)', size=14)

plt.savefig(str(path_save) +'/mean_vel.jpeg', bbox_inches='tight')
print(mannwhitneyu (df1.mean_vel,df2.mean_vel))
# with open(file_path, "a") as f:
#     f.write(f"Results for mean bout velocity:\n")
#     f.write(f"statistic={mannwhitneyu(df1.mean_vel.dropna(), df2.mean_vel.dropna()).statistic}, p-value={mannwhitneyu(df1.mean_vel.dropna(), df2.mean_vel.dropna()).pvalue}\n")

### Angle distributions

In [ ]:
#change these dfs depending on what i want to plot
angles_df1=get_angles(all_fish[all_fish['genotype']=='wt'])
angles_df2=get_angles(all_fish[all_fish['genotype']=='inj'])

#remove None from angles
angless_df1=[]
angless_df2=[]
for i in range(len(angles_df2)):
    angless_df1.append([x for x in angles_df1[i] if x is not None])
    angless_df2.append([y for y in angles_df2[i] if y is not None])

In [ ]:
pal2= {"WT": '#5D5D5D', "Mutant":'#BE4830'}

In [ ]:
ang_df_p=all_fish[['genotype', 'angles_pause']]
angles_df_p=ang_df_p.explode('angles_pause', ignore_index=True)
angles_df_p = angles_df_p[(angles_df_p['angles_pause'] >= -300) & (angles_df_p['angles_pause'] <= 300)]
angles_df_p['genotype'] = angles_df_p['genotype'].replace({'wt': 'WT', 'inj': 'Mutant'})

ax=sns.displot(angles_df_p, x=angles_df_p['angles_pause'], hue='genotype', stat="density", palette=pal2, bins=500, common_norm=False)
ax.set(xlabel='Turn angles [deg]', ylabel='Probability density')

#ax.set(title='angle distribution during pause stimulus')
plt.xlim(-120,120)
ax.savefig(str(path_save) +'/angle_probs_pause.jpeg', bbox_inches='tight')

In [ ]:
ang_df_p=all_fish[['genotype', 'angles_stim_l']]
angles_df_p=ang_df_p.explode('angles_stim_l', ignore_index=True)
angles_df_p['genotype'] = angles_df_p['genotype'].replace({'wt': 'WT', 'inj': 'Mutant'})
angles_df_p = angles_df_p[(angles_df_p['angles_stim_l'] >= -300) & (angles_df_p['angles_stim_l'] <= 300)]
ax=sns.displot(angles_df_p, x=angles_df_p['angles_stim_l'], hue='genotype', stat="density", bins=500, palette=pal2, common_norm=False)
ax.set(xlabel='Turn angles [deg]', ylabel='Probability density')
ax.set(title='Angle distribution during left stimulus')
plt.xlim(-120,120)
ax.savefig(str(path_save) +'/angle_probs_left.jpeg', bbox_inches='tight')

In [ ]:
ang_df_p=all_fish[['genotype', 'angles_stim_r']]
angles_df_p=ang_df_p.explode('angles_stim_r', ignore_index=True)
angles_df_p['genotype'] = angles_df_p['genotype'].replace({'wt': 'WT', 'inj': 'Mutant'})
angles_df_p = angles_df_p[(angles_df_p['angles_stim_r'] >= -300) & (angles_df_p['angles_stim_r'] <= 300)]
ax=sns.displot(angles_df_p, x=angles_df_p['angles_stim_r'], hue='genotype', stat="density",palette=pal2, bins=500, common_norm=False)
ax.set(xlabel='Turn angles [deg]', ylabel='Probability density')
ax.set(title='Angle distribution during right stimulus')
plt.xlim(-120,120)
ax.savefig(str(path_save) +'/angle_probs_right.jpeg', bbox_inches='tight')